**First API: Calling the search by city API to get the address of all the houses in the three cities.** The API would return the address of all the houses along with basic information about the houses.

In [ ]:
import requests
import time
areas= ['San Jose', 'San Francisco', 'Oakland']
searchResponseInfoList = list()
for area in areas:
  page = 1
  while True:
    url = "https://zillow56.p.rapidapi.com/search"
    querystring = {"location": area, "page": page}
    headers = {
        "X-RapidAPI-Key": "83f04ec7b0msh3326f127fea9304p1c9f23jsn17fc4033a01b",
        "X-RapidAPI-Host": "zillow56.p.rapidapi.com"
    }
    response_1 = requests.get(url, headers=headers, params=querystring)
    responseInfo = response_1.json()
    totalPages = responseInfo['totalPages']
    if(totalPages == page):
        break
    page = page+1
    searchResponseInfoList = searchResponseInfoList + responseInfo['results']
    time.sleep(1)
print(searchResponseInfoList)


In [ ]:
print(len(searchResponseInfoList))

2173


**Function to extract streetAddress of all the houses in each city.**

In [ ]:
def streetAddressInfo(info):
    return fetchRequestedInfo(info,['streetAddress','city','state','zipcode'])

def fetchRequestedInfo(data, keys):
    result = list()
    for info in data:
        temp = list()
        for key in keys:
            temp.append(info[key])
        result.append(", ".join(temp))
    return result
streetAddress = streetAddressInfo(searchResponseInfoList)
print(streetAddress)
print(len(streetAddress))

['3346 Prado Ln, San Jose, CA, 95148', '1876 Tradan Dr, San Jose, CA, 95132', '6382 San Anselmo Way, San Jose, CA, 95119', '1702 Prudence Way, San Jose, CA, 95116', '2314 Emerald Hills Cir, San Jose, CA, 95131', '7253 Gold Creek Way, San Jose, CA, 95120', '333 Santana Row APT 311, San Jose, CA, 95128', '2325 Cory Ave, San Jose, CA, 95128', '2151 Oakland Rd SPC 605, San Jose, CA, 95131', '2930 Mitton Dr, San Jose, CA, 95148', '2320 Homewood Dr, San Jose, CA, 95128', '2988 Grassina St UNIT 413, San Jose, CA, 95136', '2086 Foxhall Loop, San Jose, CA, 95125', '5242 Country Forge Ln, San Jose, CA, 95136', '5854 Zileman Ct, San Jose, CA, 95123', '555 Atlanta Ave, San Jose, CA, 95125', '8822 Wine Valley Cir, San Jose, CA, 95135', '1391 Sunbeam Cir #1391, San Jose, CA, 95122', '1181 Adrian Way, San Jose, CA, 95122', '1046 Liebelt Ct, San Jose, CA, 95126', '477 Mill Pond Dr #477, San Jose, CA, 95125', '6149 Countess Dr, San Jose, CA, 95129', '6240 Sager Way, San Jose, CA, 95123', '366 Pinefield

**Function to extract only the required columns from the complete details fetched by second API**

In [ ]:
def get_nested_key(data, keys):
            for key in keys:
                if key in data:
                    data = data[key]
                else:
                    return None
            return data
def buildHouseDetailsInfo(data):
    house_details_ = {
        'city': get_nested_key(data, ['address', 'city']),
        'community': get_nested_key(data, ['address', 'community']),
        'neighborhood': get_nested_key(data, ['address', 'neighborhood']),
        'state': get_nested_key(data, ['address', 'state']),
        'streetAddress': get_nested_key(data, ['address', 'streetAddress']),
        'zipcode': get_nested_key(data, ['address', 'zipcode']),
        'bedrooms': get_nested_key(data, ['bedrooms']),
        'bathrooms': get_nested_key(data, ['bathrooms']),
        'county': get_nested_key(data, ['county']),
        'datePosted': get_nested_key(data, ['datePostedString']),
        'dateSold': get_nested_key(data, ['dateSold']),
        'daysOnZillow': get_nested_key(data, ['daysOnZillow']),
        'description': get_nested_key(data, ['description']),
        'isBankOwned': get_nested_key(data, ['foreclosureTypes', 'isBankOwned']),
        'wasForeclosed': get_nested_key(data, ['foreclosureTypes', 'wasForeclosed']),
        'homeType': get_nested_key(data, ['homeType']),
        'isZillowOwned': get_nested_key(data, ['isZillowOwned']),
        'isListedByOwner': get_nested_key(data, ['isListedByOwner']),
        'lastSoldPrice': get_nested_key(data, ['lastSoldPrice']),
        'livingArea': get_nested_key(data, ['livingArea']),
        'livingAreaUnitsShort': get_nested_key(data, ['livingAreaUnitsShort']),
        'livingAreaValue': get_nested_key(data, ['livingAreaValue']),
        'longitude': get_nested_key(data, ['longitude']),
        'latitude': get_nested_key(data, ['latitude']),
        'lotAreaUnits': get_nested_key(data, ['lotAreaUnits']),
        'lotAreaValue': get_nested_key(data, ['lotAreaValue']),
        'lotSize': get_nested_key(data, ['lotSize']),
        'hdpUrl': get_nested_key(data,['hdpUrl'])
    }
    if 'hdpUrl' in house_details_.keys() and house_details_['hdpUrl'] is not None:
      house_details_['zpId'] = house_details_['hdpUrl'].strip("/").split("/")[-1].split("_")[0]
    if (('onsiteMessage' in data.keys()) and (data['onsiteMessage'] is not None) and (data['onsiteMessage']['messages'] is not None) and (len(data['onsiteMessage']['messages']) > 0)):
        if 'zestimate' in data['onsiteMessage']['messages'][0]['decisionContext'].keys():
            house_details_['zestimate'] = data['onsiteMessage']['messages'][0]['decisionContext']['zestimate']

        if 'listPrice' in data['onsiteMessage']['messages'][0]['decisionContext'].keys():
            house_details_['listPrice'] = data['onsiteMessage']['messages'][0]['decisionContext']['listPrice']
    if ('mortgageRates' in data.keys()) and (data['mortgageRates'] is not None) and ('arm5Rate' in data['mortgageRates'].keys()) and (data['mortgageRates']['arm5Rate'] is not None):
        house_details_['arm5Rate'] = data['mortgageRates']['arm5Rate']
        if data['mortgageRates']['fifteenYearFixedRate'] is not None:
            house_details_['fifteenYearFixedRate'] = data['mortgageRates']['fifteenYearFixedRate']
        if data['mortgageRates']['thirtyYearFixedRate'] is not None:
            house_details_['thirtyYearFixedRate'] = data['mortgageRates']['thirtyYearFixedRate']
    if 'resoFacts' in data.keys() and data['resoFacts'] is not None:
      if 'atAGlanceFacts' in data['resoFacts'].keys() :
        factsInfo = data['resoFacts']['atAGlanceFacts']
        lablesRequired = ['Year Built','Parking','Price/sqft']
        if factsInfo is not None:
            for info in factsInfo:
                if info['factLabel'] in lablesRequired:
                    house_details_[info['factLabel']] = info['factValue']
      house_details_['stories'] = data['resoFacts']['stories']
    if 'taxAssessedValue' in data.keys():
      house_details_['taxAssessedValue'] = data['taxAssessedValue']
    if 'taxAssessedYear' in data.keys():
      house_details_['taxAssessedYear'] = data['taxAssessedYear']

    return house_details_

**Second API: Search by Address API to fetch the complete details about a house based on its address.**

In [ ]:
import requests
import json

def callSearchAddressApi(address):
    url = "https://zillow56.p.rapidapi.com/search_address"
    querystring = {"address": address}

    headers = {
        "X-RapidAPI-Key": "83f04ec7b0msh3326f127fea9304p1c9f23jsn17fc4033a01b",
        "X-RapidAPI-Host": "zillow56.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)
    return response



In [ ]:
#Function to fetch only the required house details.
def fetchHouseDetailsInfo(data):
    # Extract details using the get_nested_key function
    house_details_ = buildHouseDetailsInfo(data)
    #print(house_details_)

    return house_details_

In [ ]:
print(len(streetAddress))
thiset = set(streetAddress)
print(type(thiset))
print(len(thiset))

2173
<class 'set'>
2169


In [ ]:
#print(streetAddress)

In [ ]:
#Calling the Details API to get the complete details of each house
import time
totalInfo = dict()
for address in streetAddress:
    response = callSearchAddressApi(address)
    if response.status_code == 200:
      totalInfo[address] = response.json()
      time.sleep(1)

In [ ]:
print(len(totalInfo))

2169


In [ ]:
#Saving the Street Addresses of all the houses in the streetAddress.json file
with open('streetAddress.json', 'w') as file:
    json.dump(totalInfo, file)

In [ ]:
#!pip install nbformat

In [ ]:
#Saving the Street Addresses of all the houses in the CompletetotalAddressInfo.json file
import nbformat
import json
from google.colab import files


with open('CompletetotalAddressInfo.json','w') as file:
  json.dump(totalInfo,file)

files.download('CompletetotalAddressInfo.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**As direct accessing of the JSON file isnt supported due to low computational power of the system. Hence we are uploading the CompletetotalAddressInfo.json file to access the complete information.**

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving CompletetotalAddressInfo.json to CompletetotalAddressInfo (1).json


In [ ]:
import json
response = uploaded['CompletetotalAddressInfo (1).json'].decode("utf-8")
totalInfo = json.loads(response)

In [ ]:
#To call the function fetch only the necessary house details of each address from the complete data
houseDetailsInfo = dict()
for address,info in totalInfo.items():
    houseDetails = fetchHouseDetailsInfo(info)
    houseDetailsInfo[address] = houseDetails

with open('CompletehouseDetailsInfoList.json', 'w') as file:
    json.dump(houseDetailsInfo, file)

In [ ]:
print(houseDetailsInfo.keys())

dict_keys(['7256 Wild Creek Dr, San Jose, CA, 95120', '524 Hobie Ln, San Jose, CA, 95127', '1000 La Terrace Cir, San Jose, CA, 95123', '1433 Lee Pl, San Jose, CA, 95131', '1726 Coraltree Pl, San Jose, CA, 95131', '5639 Le Fevre Dr, San Jose, CA, 95118', '3276 Cuesta Dr, San Jose, CA, 95148', '3052 Melchester Dr, San Jose, CA, 95132', '4110 Golf Dr, San Jose, CA, 95127', '3434 Picadilly Dr, San Jose, CA, 95118', '950 Lakeshire Ct, San Jose, CA, 95126', '2595 Hill Park Dr, San Jose, CA, 95124', '881 Byerley Ave, San Jose, CA, 95125', '2212 Summereve Ct, San Jose, CA, 95122', '3382 Madonna Dr, San Jose, CA, 95117', '1049 Obrien Ct, San Jose, CA, 95126', '775 Warring Dr APT 2, San Jose, CA, 95123', '707 River View Dr, San Jose, CA, 95111', '553 King George Ave, San Jose, CA, 95136', '2260 Constitution Dr, San Jose, CA, 95124', '736 Dalewood Ct, San Jose, CA, 95120', '125 Patterson St UNIT 306, San Jose, CA, 95112', '1140 Willow Village Sq, San Jose, CA, 95125', '484 Curie Dr, San Jose, CA,

In [ ]:
'''
import json
houseDetailsInfo = json.loads(response)
houseDetailsInfo.keys()
'''


dict_keys(['7256 Wild Creek Dr, San Jose, CA, 95120', '524 Hobie Ln, San Jose, CA, 95127', '1000 La Terrace Cir, San Jose, CA, 95123', '1433 Lee Pl, San Jose, CA, 95131', '1726 Coraltree Pl, San Jose, CA, 95131', '5639 Le Fevre Dr, San Jose, CA, 95118', '3276 Cuesta Dr, San Jose, CA, 95148', '3052 Melchester Dr, San Jose, CA, 95132', '4110 Golf Dr, San Jose, CA, 95127', '3434 Picadilly Dr, San Jose, CA, 95118', '950 Lakeshire Ct, San Jose, CA, 95126', '2595 Hill Park Dr, San Jose, CA, 95124', '881 Byerley Ave, San Jose, CA, 95125', '2212 Summereve Ct, San Jose, CA, 95122', '3382 Madonna Dr, San Jose, CA, 95117', '1049 Obrien Ct, San Jose, CA, 95126', '775 Warring Dr APT 2, San Jose, CA, 95123', '707 River View Dr, San Jose, CA, 95111', '553 King George Ave, San Jose, CA, 95136', '2260 Constitution Dr, San Jose, CA, 95124', '736 Dalewood Ct, San Jose, CA, 95120', '125 Patterson St UNIT 306, San Jose, CA, 95112', '1140 Willow Village Sq, San Jose, CA, 95125', '484 Curie Dr, San Jose, CA,

In [ ]:
files.download('CompletehouseDetailsInfoList_4.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
houseDetailsList = list()
for address,houseDetails in houseDetailsInfo.items():
    houseDetails['address'] = address;
    houseDetailsList.append(houseDetails)

In [ ]:
print(houseDetailsList[0].keys())

dict_keys(['city', 'community', 'neighborhood', 'state', 'streetAddress', 'zipcode', 'bedrooms', 'bathrooms', 'county', 'datePosted', 'dateSold', 'daysOnZillow', 'description', 'isBankOwned', 'wasForeclosed', 'homeType', 'isZillowOwned', 'isListedByOwner', 'lastSoldPrice', 'livingArea', 'livingAreaUnitsShort', 'livingAreaValue', 'longitude', 'latitude', 'lotAreaUnits', 'lotAreaValue', 'lotSize', 'hdpUrl', 'zpId', 'zestimate', 'listPrice', 'arm5Rate', 'fifteenYearFixedRate', 'thirtyYearFixedRate', 'Year Built', 'Parking', 'Price/sqft', 'stories', 'taxAssessedValue', 'taxAssessedYear', 'address'])


In [ ]:
addressListForNonZestimate = list()
for details in houseDetailsList:
  if 'zestimate' not in details.keys():
    if'zpId' not in details.keys():
      addressListForNonZestimate.append(details)
print(addressListForNonZestimate)

[{'city': None, 'community': None, 'neighborhood': None, 'state': None, 'streetAddress': None, 'zipcode': None, 'bedrooms': None, 'bathrooms': None, 'county': None, 'datePosted': None, 'dateSold': None, 'daysOnZillow': None, 'description': None, 'isBankOwned': None, 'wasForeclosed': None, 'homeType': None, 'isZillowOwned': None, 'isListedByOwner': None, 'lastSoldPrice': None, 'livingArea': None, 'livingAreaUnitsShort': None, 'livingAreaValue': None, 'longitude': None, 'lotAreaUnits': None, 'lotAreaValue': None, 'lotSize': None, 'hdpUrl': None, 'address': 'Oak, Oakland, CA, 94607'}]


**3rd API: To Fetch the estimate history of all the houses using the ZpID.**

In [ ]:
import requests


def zestimateDetailsAPi(zpId):
  url = "https://zillow56.p.rapidapi.com/zestimate_history"

  querystring = {"zpid":zpId}

  headers = {
      "X-RapidAPI-Key": "83f04ec7b0msh3326f127fea9304p1c9f23jsn17fc4033a01b",
      "X-RapidAPI-Host": "zillow56.p.rapidapi.com"
  }

  response = requests.get(url, headers=headers, params=querystring)
  return response

In [ ]:
# Filter dates for the years 2018 to 2023
def filterDate(response):
  start_timestamp = 1514764800000 #01-01-2018 in UTC format
  end_timeStamp = 1697958000000
  step = 7776000000
  if response.status_code == 200:
    new_date = response.json()['data']
    filtered_date = [item for item in new_date if start_timestamp <= item['timestamp'] <= end_timeStamp]
    return filtered_date

In [ ]:
# Filter dates in steps of 3 months between the years 2018 to 2023
def filter3monthsData(filtered_date):
  if filtered_date is None:
    return
  start_timestamp = 1514764800000
  end_timeStamp = 1697958000000
  step = 7776000000
  # Filter data for the years 2018 to 2023 and select data points every 3 months
  filtered_3monthsdate = []

  for item in filtered_date:
      if start_timestamp <= item['timestamp'] <= end_timeStamp:
          filtered_3monthsdate.append(item)
          start_timestamp += step  # Add 3 months in milliseconds (approximately)
  return filtered_3monthsdate

In [ ]:
for details in houseDetailsList:
  if 'zpId' not in details.keys() or  details['zpId'] is None:
    pass
  else:
      print(type(details['zpId']))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class

In [ ]:
response = zestimateDetailsAPi(houseDetailsList[0]['zpId'])
print(response.json())

{'data': [{'date': '2013-10', 'timestamp': 1383202800000, 'value': 758488}, {'date': '2013-11', 'timestamp': 1385798400000, 'value': 752593}, {'date': '2013-12', 'timestamp': 1388476800000, 'value': 779987}, {'date': '2014-01', 'timestamp': 1391155200000, 'value': 794601}, {'date': '2014-02', 'timestamp': 1393574400000, 'value': 797781}, {'date': '2014-03', 'timestamp': 1396249200000, 'value': 792983}, {'date': '2014-04', 'timestamp': 1398841200000, 'value': 783177}, {'date': '2014-05', 'timestamp': 1401519600000, 'value': 774103}, {'date': '2014-06', 'timestamp': 1404111600000, 'value': 769894}, {'date': '2014-07', 'timestamp': 1406790000000, 'value': 788506}, {'date': '2014-08', 'timestamp': 1409468400000, 'value': 776523}, {'date': '2014-09', 'timestamp': 1412060400000, 'value': 782767}, {'date': '2014-10', 'timestamp': 1414738800000, 'value': 798430}, {'date': '2014-11', 'timestamp': 1417334400000, 'value': 800602}, {'date': '2014-12', 'timestamp': 1420012800000, 'value': 792321}, 

In [ ]:
#Calling the API to fetch estimate history for each house
import time
zestimateApiInfo = dict()
for details in houseDetailsList:
  if 'zpId' in details.keys() and  details['zpId'] is not None:
    id = details['zpId']
    response = zestimateDetailsAPi(id)
    time.sleep(1)
    zestimateApiInfo[int(id)] = response

In [ ]:
#Loading the response into a JSON file and downloading the file
zestimateAPIResponse = dict()
for id, data in zestimateApiInfo.items():
  zestimateAPIResponse[id] = data.json()
with open('CompletezestimateAPIResponse.json','w') as file:
  json.dump(zestimateAPIResponse,file)

files.download('CompletezestimateAPIResponse.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#Calling each function
filteredDataDictonary = dict()
filtered3MonthDatadict = dict()
for id, data in zestimateApiInfo.items():
  filteredData = filterDate(data)
  filtered3MonthData = filter3monthsData(filteredData)
  filteredDataDictonary[id] = filteredData
  filtered3MonthDatadict[id] = (filtered3MonthData)

In [ ]:
#Loading the filtered response into a JSON file and downloading the file
with open('CompletefilteredZestimateInfo.json','w') as file:
  json.dump(filteredDataDictonary,file)

files.download('CompletefilteredZestimateInfo.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#Calculating the average zestimate price between the years 2018 to 2023
from collections import defaultdict

def averageZestimateData(data):
  # Create a dictionary to store the sum and count for each year
  yearly_data = defaultdict(lambda: {'sum': 0, 'count': 0})

  for item in data:
      timestamp = item['timestamp']
      value = item['value']

      # Convert the timestamp to a year
      year = int(item['date'].split('-')[0])

      # Update the sum and count for the year
      yearly_data[year]['sum'] += value
      yearly_data[year]['count'] += 1

  # Calculate the average value for each year
  yearly_averages = {}
  for year, data in yearly_data.items():
      if data['count'] > 0:
          average_value = data['sum'] / data['count']
          yearly_averages[year] = average_value

  return yearly_averages

In [ ]:
yearAverageDict = dict()
for id,data in filteredDataDictonary.items():
  if data is not None :
    yearAverages = averageZestimateData(data)
    print(yearAverages)
    yearAverageDict[id] = yearAverages

{2018: 1102467.75, 2019: 1016850.5, 2020: 982325.5833333334, 2021: 1197441.5, 2022: 1344466.6666666667, 2023: 1300266.6666666667}
{2018: 1005543.5833333334, 2019: 881885.4166666666, 2020: 982833.6666666666, 2023: 1232962.5}
{2018: 624541.0, 2019: 581247.0, 2020: 612823.1666666666, 2021: 655847.25, 2022: 689241.6666666666, 2023: 649766.6666666666}
{2020: 855094.8571428572, 2021: 944646.25, 2022: 983460.0, 2023: 979244.4444444445}
{2018: 1507159.6666666667, 2019: 1434546.75, 2020: 1525529.5833333333, 2021: 1637770.25, 2022: 1773216.6666666667, 2023: 1719066.6666666667}
{2018: 1375025.9166666667, 2019: 1259842.25, 2020: 1342874.0, 2021: 1629850.0833333333, 2022: 1798358.3333333333, 2023: 1693266.6666666667}
{2018: 1019771.5833333334, 2019: 924680.5833333334, 2020: 1001159.9166666666, 2021: 1132894.75, 2022: 1286500.0, 2023: 1057822.2222222222}
{2018: 1254732.8333333333, 2019: 1148715.25, 2020: 1186997.6666666667, 2021: 1309394.6666666667, 2022: 1479875.0, 2023: 1427155.5555555555}
{2018: 

In [ ]:
#Loading the average zestimate history response into a JSON file and downloading the file
with open('CompletefilteredZestimateInfo_Average.json','w') as file:
  json.dump(yearAverageDict,file)

files.download('CompletefilteredZestimateInfo_Average.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
len(yearAverageDict)

2168

In [ ]:
set(type(k) for k in yearAverageDict.keys())

{int}

In [ ]:
#Calling the function to calculate average zestimate price and appending these values into total details
for details in houseDetailsList:
  if 'zpId' in details.keys() and details['zpId'] is not None:
    id = details['zpId']
    yearlyAverageInfo = yearAverageDict[int(id)]
    for key,average in yearlyAverageInfo.items():
      details['Zestimate Average _'+ str(key)] = average

In [ ]:
print(houseDetailsList[0].keys())

dict_keys(['city', 'community', 'neighborhood', 'state', 'streetAddress', 'zipcode', 'bedrooms', 'bathrooms', 'county', 'datePosted', 'dateSold', 'daysOnZillow', 'description', 'isBankOwned', 'wasForeclosed', 'homeType', 'isZillowOwned', 'isListedByOwner', 'lastSoldPrice', 'livingArea', 'livingAreaUnitsShort', 'livingAreaValue', 'longitude', 'latitude', 'lotAreaUnits', 'lotAreaValue', 'lotSize', 'hdpUrl', 'zpId', 'zestimate', 'listPrice', 'arm5Rate', 'fifteenYearFixedRate', 'thirtyYearFixedRate', 'Year Built', 'Parking', 'Price/sqft', 'stories', 'taxAssessedValue', 'taxAssessedYear', 'address', 'Zestimate Average _2018', 'Zestimate Average _2019', 'Zestimate Average _2020', 'Zestimate Average _2021', 'Zestimate Average _2022', 'Zestimate Average _2023'])


In [ ]:
#Loading the complete details response into a JSON file and downloading the file
with open('CompletehouseDetailsZestimatePriceAverage_2.json','w') as file:
  json.dump(houseDetailsList,file)

files.download('CompletehouseDetailsZestimatePriceAverage_2.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#houseDetailsList

[{'city': 'San Jose',
  'community': None,
  'neighborhood': None,
  'state': 'CA',
  'streetAddress': '3382 Madonna Dr',
  'zipcode': '95117',
  'bedrooms': 3,
  'bathrooms': 2,
  'county': 'Santa Clara County',
  'datePosted': '2023-10-25',
  'dateSold': None,
  'daysOnZillow': 0,
  'description': 'This lovely move-in ready 3 bedroom, 2 bathroom home in an A+ location has been lovingly owned by the same family for over 50 years. The recently updated kitchen features stainless steel appliances, including a double oven and hood, as well as newly installed quartz countertops and an eat-in breakfast area. The formal dining room flows seamlessly into a spacious living room with a fireplace, and French doors open up to the backyard for outdoor entertaining and al fresco dining. The sleeping quarters include two guest bedrooms and a serene primary bedroom with an en-suite bathroom. The property boasts a host of desirable features, including a fireplace, front-back irrigation system, newly i

In [ ]:
#Calling the function to build the dataframe
dataFrame = buildDF(houseDetailsList, 'main-2.csv')

In [ ]:
dataFrame.columns

Index(['zpId', 'Parking', 'streetAddress', 'lotAreaUnits', 'arm5Rate',
       'daysOnZillow', 'hdpUrl', 'datePosted', 'isBankOwned', 'dateSold',
       'zipcode', 'lastSoldPrice', 'livingAreaUnitsShort', 'livingAreaValue',
       'lotSize', 'Zestimate Average _2018', 'description', 'stories',
       'livingArea', 'Zestimate Average _2023', 'latitude',
       'Zestimate Average _2021', 'isZillowOwned', 'Price/sqft',
       'wasForeclosed', 'Year Built', 'Zestimate Average _2019',
       'taxAssessedValue', 'city', 'Zestimate Average _2020', 'longitude',
       'community', 'state', 'county', 'address', 'bathrooms', 'listPrice',
       'thirtyYearFixedRate', 'zestimate', 'bedrooms', 'neighborhood',
       'isListedByOwner', 'lotAreaValue', 'fifteenYearFixedRate',
       'taxAssessedYear', 'Zestimate Average _2022', 'homeType'],
      dtype='object')

In [ ]:
dataFrame.head(5)

,zpId,Parking,streetAddress,lotAreaUnits,arm5Rate,daysOnZillow,hdpUrl,datePosted,isBankOwned,dateSold,...,thirtyYearFixedRate,zestimate,bedrooms,neighborhood,isListedByOwner,lotAreaValue,fifteenYearFixedRate,taxAssessedYear,Zestimate Average _2022,homeType
0,19846948.0,1 Attached Garage space,7256 Wild Creek Dr,Square Feet,7.291,0.0,/homedetails/7256-Wild-Creek-Dr-San-Jose-CA-95...,2023-10-25,False,1.071533e+12,...,7.641,996400.0,3.0,NaN,False,2538.0,6.936,2023.0,1.344467e+06,SINGLE_FAMILY
1,19782851.0,2 Garage spaces,524 Hobie Ln,Square Feet,7.291,0.0,/homedetails/524-Hobie-Ln-San-Jose-CA-95127/19...,2023-10-25,False,1.470355e+12,...,7.641,1006700.0,3.0,NaN,False,8579.0,6.936,2023.0,NaN,SINGLE_FAMILY
2,19831756.0,1 Carport space,1000 La Terrace Cir,sqft,7.291,0.0,/homedetails/1000-La-Terrace-Cir-San-Jose-CA-9...,2023-10-25,False,1.672358e+12,...,7.641,715300.0,2.0,NaN,False,NaN,6.936,2023.0,6.892417e+05,CONDO
3,124743687.0,1 Attached Garage space,1433 Lee Pl,Square Feet,7.291,0.0,/homedetails/1433-Lee-Pl-San-Jose-CA-95131/124...,2023-10-25,False,NaN,...,7.641,NaN,2.0,NaN,False,546.0,6.936,2023.0,9.834600e+05,TOWNHOUSE
4,19562628.0,2 Attached Garage spaces,1726 Coraltree Pl,Square Feet,7.543,0.0,/homedetails/1726-Coraltree-Pl-San-Jose-CA-951...,2023-10-25,False,NaN,...,7.676,1707000.0,5.0,NaN,False,6760.0,7.522,2023.0,1.773217e+06,SINGLE_FAMILY


In [ ]:
dataFrame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2164 entries, 0 to 2163
Data columns (total 47 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   zpId                     2163 non-null   float64
 1   Parking                  2084 non-null   object 
 2   streetAddress            2163 non-null   object 
 3   lotAreaUnits             2163 non-null   object 
 4   arm5Rate                 2149 non-null   float64
 5   daysOnZillow             2158 non-null   float64
 6   hdpUrl                   2163 non-null   object 
 7   datePosted               2156 non-null   object 
 8   isBankOwned              2163 non-null   object 
 9   dateSold                 1389 non-null   float64
 10  zipcode                  2163 non-null   float64
 11  lastSoldPrice            1389 non-null   float64
 12  livingAreaUnitsShort     2142 non-null   object 
 13  livingAreaValue          2142 non-null   float64
 14  lotSize                 

In [ ]:
dataFrame.isna().sum()

zpId                          1
Parking                      80
streetAddress                 1
lotAreaUnits                  1
arm5Rate                     15
daysOnZillow                  6
hdpUrl                        1
datePosted                    8
isBankOwned                   1
dateSold                    775
zipcode                       1
lastSoldPrice               775
livingAreaUnitsShort         22
livingAreaValue              22
lotSize                     265
Zestimate Average _2018     656
description                  14
stories                     977
livingArea                   22
Zestimate Average _2023     321
Zestimate Average _2021     568
isZillowOwned                 1
Price/sqft                  162
wasForeclosed                 1
Year Built                   86
Zestimate Average _2019     644
taxAssessedValue            436
city                          1
Zestimate Average _2020     562
longitude                     3
community                  2162
state   

In [ ]:
#Converting the dataframe into a final csv
csv_file= 'bayareadata_2.csv'
dataFrame.to_csv(csv_file,index=False)


In [ ]:
#Downloading the final csv
files.download('bayareadata_2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>